In [1]:
%load_ext autoreload
%autoreload 2

#import model
#import training
import loaders
import gamePrinter

import sys
from demoparser.demofile import DemoFile
import matplotlib.pyplot as plt
import time
import numpy as np

%matplotlib qt
%load_ext tensorboard.notebook

In [2]:
import keras
from keras import backend as K

#Check to make sure my GPU is running/is been used
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
#Import Game data into games list
games = loaders.import_games_by_directory("test_CSGO_DEM_FILES")

Loading game 0
Round  0
Round  1
Round  2
Round  3
Round  4
Round  5
Round  6
Round  7
Round  8
Round  9
Round  10
Round  11
Round  12
Round  13
Round  14
Round  15
Round  16
Round  17
Round  18
Round  19
Round  20
Round  21
Round  22
Round  23
Round  24
Round  25
Finished loading game!


In [4]:
import math

batch_size = 16
network_z, network_x, network_y = 10, 128, 128
blob_size = 2.5

def get_batches():
    #This is'nt an efficent way of flattening list lists, but it does look cool
    rounds = sum([g.m_rounds for g in games],[])
    
    #Currently just stick all rounds indiscriminitely together
    for r in rounds:
        l = max([len(p.states) for p in r.players])
        print("Generating batches from round", r.m_rid, "length", l)

        for b_ix in range(l//batch_size):  # Ignore bits at end that don't make up a whole batch
            #print("Batch", b_ix)
            Y = np.zeros((batch_size, network_z, network_x, network_y))

            # Generate batch
            for f in range(batch_size):
                i = b_ix*batch_size + f  # index of frame in round

                # Draw blob for each player
                for p_ix, p in enumerate(r.players):
                    (pos, vis, dead) = p.states[i]
                    if not dead:
                        put_heatmap(Y[f, p_ix], pos)
            
            # Grab state data for each player
            states = []
            for p_ix, p in enumerate(r.players):
                states.append(p.states[b_ix*batch_size : (b_ix+1)*batch_size])

            # Reorder axis for input to network (ix,z,x,y) -> (ix,x,y,z)
            Y = np.moveaxis(Y, 1, -1)
            yield Y, states

In [5]:
# Get a batch of frame# Put a blob in the required place of a 2d matrix
# Can do this way faster, but it's fine for now..
def put_heatmap(heatmap, center):
    center_x, center_y = center
    
    # Rescale ..
    x1, x2, y1, y2 = (-2600., 2100., -1200., 3200.)

    center_x = (center_x - x1) / (x2 - x1) * network_x
    center_y = (center_y - y1) / (y2 - y1) * network_y
    
    height, width = heatmap.shape

    th = 4.6052
    
    delta = math.sqrt(th * 2)

    # Vectorize
    sigma = blob_size
    x0 = int(max(0, center_x - delta * sigma))
    y0 = int(max(0, center_y - delta * sigma))

    x1 = int(min(width - 1, center_x + delta * sigma))
    y1 = int(min(height - 1, center_y + delta * sigma))

    exp_factor = 1 / 2.0 / sigma / sigma
    arr_heatmap = heatmap[y0:y1 + 1, x0:x1 + 1] # Not nessasary unless we have > 1 blob per input
    y_vec = (np.arange(y0, y1 + 1) - center_y) ** 2  # y1 included
    x_vec = (np.arange(x0, x1 + 1) - center_x) ** 2
    xv, yv = np.meshgrid(x_vec, y_vec)
    arr_sum = exp_factor * (xv + yv)
    arr_exp = np.exp(-arr_sum)
    arr_exp[arr_sum > th] = 0
    
    heatmap[y0:y1 + 1, x0:x1 + 1] = np.maximum(arr_heatmap, arr_exp)


In [6]:
# Do teh learning!

from model import unet
#pretrained_weights="csgo_zero.h5"
model = unet(input_size=(128,128,10))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/alec/Documents/CSGOProj/model.py:59: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


In [7]:
num_epochs = 100
debug = False

def train_network():
    step = 0
    print_param = gamePrinter.print_single_frame(np.ones((160,160,10)));
    for e in range(num_epochs):
        
        btchs = get_batches()
        # Just dump first batch pretty much

        Y0, _ = next(btchs)

        # Initial state is just the very first frame of the game,
        # which is known to both sides anyway(!)
        # After the first loop, it'll be a prediction of the first frame in the batch.
        c_frame = Y0[-1]

        # Frame index for input vs output.
        # k(ix) = global knowledge from frame ix

        # input (X)       | output (Y)
        # ----------------------------
        # 0               | 1
        # pred(0) + k(1)  | 2
        # pred(x1) + k(2) | 3
        # pred(x2) + k(3) | 4

        #while True:
        for Y, states in btchs:
            # Each loop here creates a new training batch and updates the network based on it.

            # Make batch input
            X = np.empty((batch_size, network_x, network_y, network_z))

            for f_ix in range(batch_size):
                # Add current frame to batch (copying)
                # This way, X always lags behind Y by one frame.
                X[f_ix,:] = c_frame

                # Predict next frame (f_ix) based on current one (expanding and reducing dimensions cause stuff)
                c_frame = model.predict(np.expand_dims(c_frame, axis=0))[0]

                # Update with any knowledge we have. (i.e. player visible or dead)
                #ix = starting_frame + f_ix  # current frame index

                for p_id in range(network_z):
                    _, is_spotted, is_dead = states[p_id][f_ix]
                    if p_id < 5 or is_spotted or is_dead:
                        # If player is dead or visible, we now know what it's doing...
                        # update it's layer to the ground truth. (of what will be effectively the previous frame)
                        c_frame[:,:,p_id] = Y[f_ix,:,:,p_id]                
            
            

            # Train model!!! on our newly created X and Y batches (:D)
            loss = model.train_on_batch(X, Y)
            #import pdb; pdb.set_trace()
            print_param = gamePrinter.print_single_frame(c_frame, print_param[0], print_param[1]);
            if step % 100 == 0:
                print(loss)
                
            step += 1
          
        
        #import pdb; pdb.set_trace() #check that frame makes sense using print_frame()
    #model.save('csgo_{}.h5'.format(e))  # creates a HDF5 file 'my_model.h5'
    if e % 3 == 0:
        model.save('csgo_zero.h5')

train_network()

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

Generating batches from round 0 length 1392
Instructions for updating:
Use tf.cast instead.


<Figure size 432x288 with 0 Axes>

[0.69565094, 0.4189242]


...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir logs/scalars